# Demo: Getting Started with the CAS Server

### 1. Import Packages

Visit the documentation for the SWAT [(SAS Scripting Wrapper for Analytics Transfer)](https://sassoftware.github.io/python-swat/index.html) package.

In [ ]:
import swat
import pandas as pd

### 2. Connect to the CAS Server


To connect to the CAS server, you need
- the host name 
- the port number 
- authentication

Visit the documentation [Getting Started with SAS® Viya® for Python](https://documentation.sas.com/doc/en/pgmsascdc/v_016/caspg3/titlepage.htm?homeOnFail) for more information about connecting to CAS.

**Note**: Be aware that connecting to the CAS server can be implemented in various ways, so you might need to see your system administrator about how to make a connection. Please follow company policy regarding authentication. ***Never place your password in plain text. This course adds the password in plain text for training purposes only.***



a. Make a connection to the CAS server and name the variable **conn**.

In [ ]:
conn = swat.CAS('server.demo.sas.com', 30571, 'student', 'Metadata0')

b. Print the value of the **conn** variable. Notice that it contains connection information to the CAS server. A random session name and session ID is assigned.

In [ ]:
print(conn)

c. Print the type of the **conn** variable. Notice that it's a **CAS** connection object.

In [ ]:
type(conn)

### 3. Explore the CAS Session Using CAS Actions

a. Use the [session.listSessions](https://go.documentation.sas.com/doc/en/pgmsascdc/v_016/caspg/cas-session-listsessions.htm?homeOnFail) action to display a list of CAS sessions.

In [ ]:
conn.session.listSessions()

b. You can also specify the action without the action set. Here, the connection object **conn** is specified, followed by the CAS action listSesssions. Typically, throughout this course, only the action name is used. Execute the cell. Notice that the results are identical.

In [ ]:
conn.listSessions()

c. Use the [sessionProp.listSessOpts](https://go.documentation.sas.com/doc/en/pgmsascdc/v_016/caspg/cas-sessionprop-listsessopts.htm) action to display the session options and session values.

In [ ]:
conn.listSessOpts()

d. Modify CAS session options using the [sessionProp.setSessOpt](https://go.documentation.sas.com/doc/en/pgmsascdc/v_016/caspg/cas-sessionprop-setsessopt.htm) action with the name of the option to modify as a parameter. Here the **timeout** parameter is specified with a new value of *3600*. Then the listSessOpts action is executed to view the new value of the **timeout** option.

In [ ]:
conn.setSessOpt(timeout = 3600)
conn.listSessOpts()

e. Use the [builtins.actionSetInfo](https://go.documentation.sas.com/doc/en/pgmsascdc/v_016/caspg/cas-builtins-actionsetinfo.htm?homeOnFail) action to display all of the available action sets. By default, actionSetInfo shows all loaded action sets. 

**Note:** Think of an action set as a package, and actions within an action set as methods.

In [ ]:
conn.actionSetInfo()

f. You can use the help function to see the available actions within an action set. Within the help function, the builtIns action set is specified. View all the actions available within the builtins action set.

In [ ]:
help(conn.builtIns)

g. In the actionSetInfo action add the all parameter with the value *True* to see all available action sets. Notice that most action sets are not loaded by default.

In [ ]:
conn.actionsetInfo(all = True)

### 4. Working with CAS Tables - Comparing the Pandas API and CAS Actions in the SWAT Package.

Create a reference to a CAS table using the [CASTable](https://sassoftware.github.io/python-swat/generated/swat.cas.table.CASTable.html#swat.cas.table.CASTable) method in the variable **tbl**. Print the value of the new **tbl** variable and its type. Notice that it's a reference to an in-memory table on the CAS server and the object type is **CASTable**. Remember, this variable does not store a table, only a reference to a CAS table.

In [ ]:
tbl = conn.CASTable('cars', caslib = 'casuser')

display(type(tbl), tbl)

#### Using the SWAT Package Pandas API on a CAS Table

a. Execute the [head](https://sassoftware.github.io/python-swat/generated/swat.cas.table.CASTable.head.html#swat.cas.table.CASTable.head) method on the CAS table. The CAS server processes the data and then returns five rows to the client.

In [ ]:
tbl.head()

b. View the object type returned to the client from the head method. Notice that it's a **SASDataFrame** object.

In [ ]:
df = tbl.head()
type(df)

**Notes**:
A **SASDataFrame** object
- is data that is local on the client. Remember that CAS can hold larger data than your local computer can handle.
- is a subclass of a **pandas.DataFrame**. You can work with **SASDataFrame** objects as you normally do a **pandas.DataFrame**.

c. Use the Pandas plot method on the client-side **SASDataFrame** object to create a bar chart. Notice you can easily work with objects returned from the CAS server with familiar Python packages.

In [ ]:
df.plot(kind = 'bar', x = 'Model', y = 'MSRP', 
        figsize = (8,6));

d. Use the [value_counts](https://sassoftware.github.io/python-swat/generated/swat.cas.table.CASColumn.value_counts.html#swat.cas.table.CASColumn.value_counts) method on a **CASTable** object to return the number of unique values of a CAS table column. View the object type returned from the CAS server. Notice that the CAS server returns a **Series** object to the client.

In [ ]:
s = (tbl
     .Type
     .value_counts())

display(type(s), s)

e. Use the value_counts method on a CAS table to return a **Series** object to the client. Then use the Pandas plot method on the client to plot the results. The value_counts method summarizes the CAS table on the CAS server and returns a **Series** object to the client. Then the Pandas plot method visualizes the **Series** on the client.

In [ ]:
(tbl
 .Type
 .value_counts(normalize = True)
 .plot(kind = 'bar', figsize = (8,6)));

f. Use the [shape](https://sassoftware.github.io/python-swat/generated/swat.cas.table.CASTable.shape.html#swat.cas.table.CASTable.shape) attribute to show the dimensions of the CAS table. Check the object type returned from the CAS server. Notice that the shape attribute returns a **tuple** to the client.

In [ ]:
shape = tbl.shape

display(shape, type(shape))

**Note**:
- A **CASTable** object is a reference to an in-memory table on the CAS server.
- The SWAT package API contains many of the methods defined by the **pandas.DataFrame** object. Using these methods from the SWAT API will typically return a **CASTable**, **CASColumn**, **pandas.DataFrame**, **SASDataFrame**, or **pandas.Series** object to the client.
- Once summarized data is returned to the client, you can work with it as you normally would in Python.

#### Using a CAS Action on a CAS Table

a. Execute the [table.fetch](https://go.documentation.sas.com/doc/en/pgmsascdc/v_016/caspg/cas-table-fetch.htm?homeOnFail) action on a CAS table to return five rows. Notice that there are a variety of ways to execute an action.

In [ ]:
## Method 1
tbl.fetch(to = 5)

## Method 2
#tbl.table.fetch(to = 5)

## Method 3
#conn.fetch(table = tbl, to = 5)

## Method 4
#conn.fetch(table = {'name':'cars','caslib':'casuser'}, to = 5)

b. View the object returned to the client from a CAS action. Notice that it's a **CASResults** object. A **CASResults** object is simply a Python dictionary with some additional attributes and methods.

In [ ]:
type(tbl.fetch(to = 5))

c. Store the **CASResults** object in a variable named **cr**. View the available keys in the **CASResults** object using the keys method. Notice that the keys method returns a single key, *Fetch*. If you look at the output of the previous cell, you can see that the key is located above the object.

In [ ]:
cr = tbl.fetch(to = 5)
cr.keys()

d. Call the *Fetch* key in the **CASResults** object to return the value the key holds. View the type and value of the returned object. Notice that the **CASResults** object contains a **SASDataFrame**. 

In [ ]:
display(type(cr['Fetch']), 
        cr['Fetch'])

e. Call the *Fetch* key from the **CASResults** object to return the **SASDataFrame**. Once you have the **SASDataFrame**, you can work with it as you would a **pandas.DataFrame**. Here, the Pandas plot method is executed.

In [ ]:
(cr['Fetch']
 .plot(kind = 'bar', x = 'Model',y = 'MSRP', 
       figsize=(8,6)));

**Notes**:
- CAS actions interact with the CAS server and return a **CASResults** object. 
- A **CASResults** object is simply an ordered Python dictionary with a few extra methods and attributes added.
- **CASResults** objects are local on the client.
- Although all CAS actions return a **CASResults** object, there are no rules about how many keys are contained in the object or what objects are returned.

### 5. View the Execution of the CAS API

a. To view what the CAS API is executing on the CAS server, you can enable tracing of actions by turning on the **trace_actions** and **trace_ui_actions** options.

In [ ]:
swat.options.cas.trace_actions = True      
swat.options.cas.trace_ui_actions = True

b. Execute the SWAT head method on the **CASTable** object. Notice that the cell shows what actions were executed on the CAS server. In this example, the head method executes the fetch action on the CAS server.

In [ ]:
tbl.head()

c. Execute the value_counts method on the **CASTable** object. Notice that the cell shows that the columnInfo and freq actions were executed on the CAS server to produce similar results to the value_counts method.

In [ ]:
(tbl
 .Type
 .value_counts())

d. Execute the fetch CAS action on the **CASTable** object. Notice that the output shows that the fetch action was executed on the CAS server.

In [ ]:
tbl.fetch(to = 5)

e. Set the **trace_actions** and **trace_ui_actions** options to the default *False* value.

In [ ]:
swat.options.cas.trace_actions = False      
swat.options.cas.trace_ui_actions = False

### 6. Terminate the CAS Session

It's best practice to always terminate the CAS session when you are done.

In [ ]:
conn.terminate()